In [1]:
from src.models.level_kv_div import binaryTrainer, utils, network
from torch.utils.data import DataLoader

import pandas as pd 
import torch

In [2]:
# # train_dataset = binaryTrainer.SparseDataset('../data/processed/sample/train.csv', '../data/processed/sample/train.level.grading', balanced=True, random_neg=False)
# # dev_dataset = binaryTrainer.SparseDataset('../data/processed/sample/dev.csv', '../data/processed/sample/dev.level.grading', balanced=True, random_neg=False)
# train_dataset = binaryTrainer.LargeSparseDataset('../data/sift/siftsmall/siftsmall_learn.csv', 100, balanced=True, random_neg=True)
# dev_dataset = binaryTrainer.LargeSparseDataset('../data/sift/siftsmall/siftsmall_base.csv', 100, balanced=True, random_neg=False)


In [3]:
# torch.save(train_dataset, '../data/sift/siftsmall/siftsmall_learn.binaryTrainer.LargeSparseDataset')
# torch.save(dev_dataset, '../data/sift/siftsmall/siftsmall_base.binaryTrainer.LargeSparseDataset')
train_dataset = torch.load('../data/sift/siftsmall/siftsmall_learn.binaryTrainer.LargeSparseDataset')
dev_dataset = torch.load('../data/sift/siftsmall/siftsmall_base.binaryTrainer.LargeSparseDataset')

In [4]:
model = network.SiameseNet(network.EmbeddingNet())
learning_rate = 4e-3
batch_size = 32768
num_epoches = 3

verbose = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
weight_decay = 1e-6
log_epoch = 3

In [5]:
# optimizer
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)
dev_loader = DataLoader(dev_dataset, shuffle=False, batch_size=batch_size, pin_memory=True)
model = model.to(device)

In [6]:
best_avg_val_kl_loss, best_model, model = binaryTrainer.train_with_eval(train_loader, dev_loader, model, optimizer, num_epoches, log_epoch, verbose, device)

epoch [3]/[3] training loss: 0.000009 avg_val_margin_loss: 0.0001 train_accuracy:  0.61 val_accuracy:  0.50 


In [7]:
best_avg_val_kl_loss

5.0551664121080155e-05

In [8]:
model = model.cpu()

In [9]:
x = torch.from_numpy(pd.read_csv('../data/sift/siftsmall/siftsmall_base.csv', header=None).to_numpy()).float()
loss_param = 1, 3, 4, 6
k = 100

In [10]:
(margin_res, margin_measure_confusion), (linear_search_res, linear_search_confusion), (pred_dist) = binaryTrainer.evaluate_results(x, best_model, k, loss_param, cache_dist=True)


pred_dist is cached!


In [11]:
margin_res

{'accuracy': 0.76697594,
 'recall': 0.998919801980198,
 'precision': 0.041501427547520744,
 'f1-score': 0.07969194901277334}

In [12]:
linear_search_res

{'accuracy': 0.9868869,
 'recall': 0.3508366336633663,
 'precision': 0.3508366336633663,
 'f1-score': 0.3508366336633663}